Importing necessary dependencies

In [ ]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import PIL #The Python Imaging Library (PIL) adds image processing capabilities to your Python interpreter
import tensorflow as tf
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Cross validation function with model creation and data processing inside. Data processing is inside the network. Data augmentation layers do not activate at inference time.

In [ ]:
        
def crossvalidation(data_dir, img_size, batch_size, folds, epochs):
    val_acc_list = []
    train_acc_list = []
    dataset = keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(img_size[0],img_size[1]),
        batch_size=batch_size)
    num_classes = len(dataset.class_names)
    print(dataset.class_names)
    ds_size = tf.data.experimental.cardinality(dataset).numpy()
    print(ds_size)
    dataset = dataset.shuffle(buffer_size=ds_size)
    for fold in range(folds):
        model = Sequential([
            layers.experimental.preprocessing.RandomTranslation(height_factor=(-0.1,0.1),
                                                                  width_factor=(-0.1,0.1),
                                                                  fill_mode="reflect",
                                                                 input_shape=img_size),
            layers.experimental.preprocessing.RandomRotation(factor=(-0.05,0.05), fill_mode="reflect"),
            layers.experimental.preprocessing.Rescaling(1./255),
            layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x)),
            layers.Lambda(lambda x: tf.image.adjust_contrast(x, 10)),
            layers.Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(32, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(128, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Dropout(0.2),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dense(num_classes)
            ])
        model.summary()
        print(f"########## FOLD {fold + 1} ##########")
        train_ds = dataset.shard(folds, index=fold)
        for i in range(folds - 2):
            train_ds = train_ds.concatenate(dataset.shard(folds, index=np.mod(fold+i+1,folds)))
            print("concatenated")
        val_ds = dataset.shard(folds, index=np.mod(fold+folds-1, folds))
        print(tf.data.experimental.cardinality(train_ds).numpy())
        print(tf.data.experimental.cardinality(val_ds).numpy())
        
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
        history = model.fit(
                  train_ds,
                  validation_data=val_ds,
                  epochs=epochs
        )
        val_acc_list.append(history.history["val_accuracy"])
        train_acc_list.append(history.history["accuracy"])
        del model
        tf.keras.backend.clear_session()
        tf.compat.v1.reset_default_graph()
    return train_acc_list, val_acc_list 


Basic data parameters

In [ ]:
img_size = (128, 128, 3)
batch_size = 32
epochs = 15
datadir ="images_train_3"

Script to generate diferent k folds. range(1) means we only do 1 kfold. This was implemented as a test.

In [ ]:
train_acc_hist = []
val_acc_hist = []

for i in range(1):
    train_accs, val_accs = crossvalidation(data_dir=datadir,
                        img_size=img_size,
                        batch_size=batch_size,
                        folds=5,
                        epochs=epochs)
    print(f"############### TRIAL {i + 1} ##############")
    train_acc_hist.append(train_accs)
    val_acc_hist.append(val_accs)

Calculations to obtain the mean accuracy for training and validation between the diferent folds

In [ ]:
mean_train_acc_hist = []
std_train_acc_hist = []
mean_val_acc_hist = []
std_val_acc_hist = []
for trial in train_acc_hist:
    for i in range(len(trial[0])):
        avg_acc_epoch = []
        for fold in trial:
            avg_acc_epoch.append(fold[i])
        mean_train_acc_hist.append(np.mean(avg_acc_epoch))
        std_train_acc_hist.append(np.std(avg_acc_epoch))
        

# Do the same for validation accuracy
for trial in val_acc_hist:
    for i in range(len(trial[0])):
        avg_acc_epoch = []
        for fold in trial:
            avg_acc_epoch.append(fold[i])
        mean_val_acc_hist.append(np.mean(avg_acc_epoch))
        std_val_acc_hist.append(np.std(avg_acc_epoch))
print(f"Train mean final acc: {np.mean(mean_train_acc_hist[-4:])}with std {np.std(mean_train_acc_hist[-4:])}")
print(f"Validation mean final acc: {np.mean(mean_val_acc_hist[-4:])} with std {np.std(mean_val_acc_hist[-4:])}")

Accuracy figures

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1, 1, 1)
ax.errorbar(np.arange(1, epochs + 1),
            mean_train_acc_hist,
            label="Mean Trainning set accuracy",
            yerr = std_train_acc_hist)
ax.errorbar(np.arange(1, epochs + 1),
        mean_val_acc_hist,
        label="Mean Validation set accuracy",
        yerr = std_val_acc_hist)
plt.xticks(range(epochs+1))
ax.set_xlabel("Epochs")
ax.set_ylabel("Accuracy")
plt.legend()
fig.savefig("foo.png")
plt.show()

Train definitive model with the whole training set.

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(img_size[0], img_size[1]),
        batch_size=batch_size)

model = Sequential([
            layers.experimental.preprocessing.RandomTranslation(height_factor=(-0.1,0.1),
                                                                  width_factor=(-0.1,0.1),
                                                                  fill_mode="reflect",
                                                                 input_shape=img_size),
            layers.experimental.preprocessing.RandomRotation(factor=(-0.05,0.05), fill_mode="reflect"),
            layers.experimental.preprocessing.Rescaling(1./255),
            layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x)),
            layers.Lambda(lambda x: tf.image.adjust_contrast(x, 10)),
            layers.Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(32, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(128, 3, padding='same', activation='relu'),
            layers.MaxPooling2D(),
            layers.Dropout(0.2),
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dense(num_classes)
])

model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy']
)
history = model.fit(
          dataset,
          epochs=epochs
)
#serialize to JSON
model_json=model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)
#serialize weights to hdf5
model.save_weights("model.h5")
print("saved model to disk")
model.save('modeltotal.h5')